# Uporaba SQLite v Pythonu

Za priklop na bazo SQLite v Pythonu uporabimo vgrajeni modul `sqlite3`.

In [1]:
import sqlite3

# testna baza
baza = "primer.db"

Ustvarimo tabelo in jo napolnimo.

In [2]:
with sqlite3.connect(baza) as con:
    cur = con.cursor()   # "odzivnik" za pregledovanje poizvedbe
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INTEGER PRIMARY KEY, Name TEXT, Price INTEGER)")
    cur.execute("INSERT INTO Cars VALUES (1, 'Audi', 52642)")
    cur.execute("INSERT INTO Cars VALUES (2, 'Mercedes', 57127)")
    cur.execute("INSERT INTO Cars VALUES (3, 'Skoda', 9000)")
    cur.execute("INSERT INTO Cars VALUES (4, 'Volvo', 29000)")
    cur.execute("INSERT INTO Cars VALUES (5, 'Bentley', 350000)")
    cur.execute("INSERT INTO Cars VALUES (6, 'Citroen', 21000)")
    cur.execute("INSERT INTO Cars VALUES (7, 'Hummer', 41400)")
    cur.execute("INSERT INTO Cars VALUES (8, 'Volkswagen', 21600)")

Povezavo si lahko tudi shranimo in večkrat uporabimo.

In [3]:
con = sqlite3.connect(baza)

Primer: Katere tabele so v bazi?

In [4]:
with con:
    cur = con.cursor()
    res = cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")
    print(res.fetchall())

[('Cars',)]


Primer: Katere vrstice so v tabeli?

In [12]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    print(cur.fetchall())

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600)]


Ko kurzor enkrat vrne podatke, se "izčrpa".

In [6]:
cur.fetchall()

[]

Z `with` smo naredili transakcijo - ob uspešnem koncu se je potrdila, sicer bi se preklicala.

In [7]:
with con:
    cur.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")
    cur.execute("INSERT INTO Cars VALUES (9, 'Ferrari', 192000)")

IntegrityError: UNIQUE constraint failed: Cars.Id

Transakcije lahko potrjujemo in preklicujemo tudi sami.

In [13]:
cur.execute("INSERT INTO Cars VALUES (9, 'Hyundai', 19200)")

In [16]:
with sqlite3.connect(baza) as con2: # uporabimo novo povezavo
    cur2 = con2.cursor()
    cur2.execute("SELECT * from Cars")
    print(cur2.fetchall())

[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (9, 'Hyundai', 19200)]


In [11]:
con.rollback()

In [15]:
con.commit()

Primer: Poizvedba z iteracijo po 'odzivniku'.

In [18]:
with con:
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    prvi = cur.fetchone()
    print("Prvi:", prvi)
    for podatek in cur:
        print(podatek)
        if podatek[0] == 5:
            break
    print(cur.fetchall())

Prvi: (1, 'Audi', 52642)
(2, 'Mercedes', 57127)
(3, 'Skoda', 9000)
(4, 'Volvo', 29000)
(5, 'Bentley', 350000)
[(6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (9, 'Hyundai', 19200)]


Primer interaktivne poizvedbe,  uporaba znaka `?`.

In [21]:
with con:
    cur = con.cursor()
    kaj = input("Cena katerega vozila te zanima:")
    cur.execute("SELECT Name, Price FROM Cars WHERE Name LIKE ?", (kaj,))
    print(cur.fetchall())

Cena katerega vozila te zanima:V%
[('Volvo', 29000), ('Volkswagen', 21600)]


Primer interaktivne poizvedbe, uporaba `:ključ`, preverjanje rezultata.

In [24]:
with con:
    cur = con.cursor()
    kaj = input("Katero vozilo te zanima:").strip()
    cur.execute("SELECT Price FROM Cars WHERE Name LIKE :ime", {"ime": kaj})
    rez = cur.fetchone()
    if rez is None:
        print("Ni takega vozila ({0}).".format(kaj))
    else:
        print("Cena vozila {0} je {1}.".format(kaj, rez[0]))

Katero vozilo te zanima:Volvo
Cena vozila Volvo je 29000.


Primer: večkratno vstavljanje.

In [25]:
cars = [
    (10, 'Audi2', 52643),
    (11, 'MercedesX', 57642),
    (12, 'Škoda', 9000),
    (13, 'Volvek', 29000),
    (14, 'Bentley7', 350000),
    (15, 'HummerU', 41400),
    (16, 'VolkswagenPassat', 21600)
]
print("Vstavljamo: ", cars)
with con:
    cur = con.cursor()
    cur.executemany("INSERT INTO Cars VALUES (?, ?, ?)", cars)
    cur.execute("SELECT * from Cars")
    print("V tabeli imamo:")
    print(cur.fetchall())

Vstavljamo:  [(10, 'Audi2', 52643), (11, 'MercedesX', 57642), (12, 'Škoda', 9000), (13, 'Volvek', 29000), (14, 'Bentley7', 350000), (15, 'HummerU', 41400), (16, 'VolkswagenPassat', 21600)]
V tabeli imamo:
[(1, 'Audi', 52642), (2, 'Mercedes', 57127), (3, 'Skoda', 9000), (4, 'Volvo', 29000), (5, 'Bentley', 350000), (6, 'Citroen', 21000), (7, 'Hummer', 41400), (8, 'Volkswagen', 21600), (9, 'Hyundai', 19200), (10, 'Audi2', 52643), (11, 'MercedesX', 57642), (12, 'Škoda', 9000), (13, 'Volvek', 29000), (14, 'Bentley7', 350000), (15, 'HummerU', 41400), (16, 'VolkswagenPassat', 21600)]


Uporaba generatorja vrstic `sqlite3.Row`. Namesto $n$-terk za vrstice vračamo objekte.

In [31]:
with con:
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    cur.execute("SELECT * from Cars")
    for vrstica in cur:
        print(vrstica["Name"])

ProgrammingError: Cannot operate on a closed database.

Ko povezave več ne potrebujemo, jo zapremo.

In [30]:
con.close()